<a href="https://colab.research.google.com/github/EdonMars/edoardonati.github.io/blob/main/Delivery_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VARIABILI INIZIALI**

In [ ]:
# Numero di ristoranti
num_restaurants = 5

# Numero di punti di consegna
num_delivery_points = 7

# Area massima di distribuzione dei ristoranti
max_area_rest = 800

# Area massima di distribuzione delle consegne
max_area_del = 1500

# Distanza minima tra i punti di consegna
min_distance_del = 200

#Velocità media del fattorino in Km/H
velocita_media_kmh = 15



**GENERAZIONE DEI RISTORANTI E CALCOLO DEL PERCORSO PIU' EFFICIENTE DA SEGUIRE PER RACCOGLIERE IL CIBO**

In [ ]:
import itertools
import random
import math
from math import sqrt

# Coordinate casuali dei ristoranti nell'area di raccolta in metri quadrati
restaurants_coords = [(random.uniform(0, max_area_rest), random.uniform(0, max_area_rest)) for i in range(num_restaurants)]

# Calcola le coordinate del punto centrale
x_center = sum(coord[0] for coord in restaurants_coords) / num_restaurants
y_center = sum(coord[1] for coord in restaurants_coords) / num_restaurants
center = (x_center, y_center)

# Calcola la distanza di ogni ristorante dal punto centrale
distances = [round(math.sqrt((coord[0]-x_center)**2 + (coord[1]-y_center)**2),2) for coord in restaurants_coords]

**GENERA LE COORDINATE DEI PUNTI DI CONSEGNA**

In [ ]:
# Coordinate casuali dei punti di consegna nell'area di consegna in metri quadrati
delivery_points_coords = [(random.uniform(0, max_area_del), random.uniform(0, max_area_del)) for i in range(num_delivery_points)]

# Verifica che la distanza tra i punti di consegna sia maggiore della distanza minima in metri

while True:
    too_close = False
    for i in range(num_delivery_points):
        for j in range(i + 1, num_delivery_points):
            dist = ((delivery_points_coords[i][0]-delivery_points_coords[j][0])**2 + (delivery_points_coords[i][1]-delivery_points_coords[j][1])**2)**0.5
            if dist < min_distance_del:
                too_close = True
                # Ricostruzione delle coordinate casuali
                delivery_points_coords = [(random.uniform(0, max_area_del), random.uniform(0, max_area_del)) for i in range(num_delivery_points)]
                break
        if too_close:
            break
    if not too_close:
        break

**DEFINISCO CINQUE FUNZIONI, TRE (NEAREST_NEIGHBOR, DISTANCE E CALCULATE_DISTANCE_FROM_MATRIX) CHE VENGONO UTILIZZATE NELLA FUNZIONE PRINCIPALE, TSP, CHE CALCOLA IL PERCORSO PIU' BREVE DAL PUNTO DI PARTENZA START. LA FUNZIONE USA LA FORZA BRUTA PER IL CALCOLO SU PERCORSI CON 10 TAPPE O MENO, MENTRE USA IL NEAREST_NEIGHBOR PER PERCORSI PIU' LUNGHI PER OTTIMIZZARE IL TEMPO DI CALCOLO. TROVA_COORD_PIU_VICINA SERVE INVECE PER CALCOLARE SUCCESSIVAMENTE IL TEMPO DI RITORNO**

In [ ]:
def trova_coord_piu_vicina(coord_list, coord_tupla):
    """
    Trova l'elemento della lista di coordinate che ha la distanza più breve dalla tupla di coordinate data.
    :param coord_list: lista di coordinate in formato [(x1,y1), (x2,y2), ...]
    :param coord_tupla: tupla di coordinate in formato (x,y)
    :return: la coordinata più vicina alla tupla data
    """
    distanza_minima = float('inf')
    coord_piu_vicina = None
    for coord in coord_list:
        distanza = sqrt((coord[0] - coord_tupla[0])**2 + (coord[1] - coord_tupla[1])**2)
        if distanza < distanza_minima:
            distanza_minima = distanza
            coord_piu_vicina = coord
    return coord_piu_vicina




def distance(coord1, coord2):
    """
    Calcola la distanza euclidea tra due coordinate.

    :param coord1: la prima coordinata
    :param coord2: la seconda coordinata
    :return: la distanza tra le due coordinate
    """
    return ((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2) ** 0.5



def nearest_neighbor(dist_matrix):
    """
    Applica l'algoritmo del nearest neighbor per trovare un percorso che visita tutti i nodi una sola volta.

    :param dist_matrix: la matrice delle distanze tra i nodi
    :param start: il nodo di partenza
    :return: un percorso che visita tutti i nodi una sola volta (come lista di punti)
    """
    num_nodes = len(dist_matrix)
    visited = [False] * num_nodes
    visited[0] = True
    path = [0]
    current_node = 0
    while len(path) < num_nodes:
        min_distance = float('inf')
        min_node = None
        for i in range(num_nodes):
            if not visited[i] and dist_matrix[current_node][i] < min_distance:
                min_distance = dist_matrix[current_node][i]
                min_node = i
        visited[min_node] = True
        path.append(min_node)
        current_node = min_node
    return path



def calculate_distances_from_matrix(dist_matrix, path):
    # Crea la lista delle distanze tra i punti del percorso
    path_distances = []
    for i in range(len(path) - 1):
        distance = dist_matrix[path[i]][path[i+1]]
        path_distances.append(distance)

    return path_distances



def tsp(coords_list, start):
    """
    Trova il percorso ottimo per visitare tutti i punti indicati nella matrice delle distanze, date una lista di coordinate ed un punto di partenza esterno alla lista.

    :param coords_list: lista delle coordinate dei punti
    :param start: punto di partenza, esterno alla lista
    :return: il percorso ottimo e la distanza tra i vari punti del percorso
    """

    coords_and_start = [start] + coords_list

    num_points = len(coords_and_start)

    # Calcolo della matrice delle distanze
    dist_matrix = [[distance(coords_and_start[i], coords_and_start[j]) for j in range(num_points)] for i in range(num_points)]

    if num_points > 10:
        # Nearest neighbor
        path = nearest_neighbor(dist_matrix)

    else:
        # Calcolo di tutti i possibili percorsi
        all_paths = itertools.permutations(range(num_points))


        # Calcolo della lunghezza di tutti i possibili percorsi
        all_paths_lengths = {}
        
        for path in all_paths:
            if coords_and_start[path[0]] == start:
                path_length = sum(dist_matrix[path[i]][path[i + 1]] for i in range(num_points - 1))
                all_paths_lengths[path] = path_length

        # Trovare il percorso più breve
        shortest_path = min(all_paths_lengths, key=all_paths_lengths.get)
        path = list(shortest_path)

    path_distances = calculate_distances_from_matrix(dist_matrix, path)

    return path, path_distances

In [ ]:
def random_delivery_time():
    r = random.random()
    if r < 0.95:
        return random.randint(120, 180)
    else:
        return random.randint(240, 300)

def random_pickup_time():
    r = random.random()
    if r < 0.95:
        return random.randint(60, 120)
    else:
        return random.randint(180, 240)




# velocità media in m/s
velocita_media_ms = (velocita_media_kmh * 1000 / 3600)

# lista delle distanze in metri tra ristoranti
distanze_rest = tsp(restaurants_coords, center)[1]

# coordinate dell'ultimo ristorante
ultimo_pickup = restaurants_coords[tsp(restaurants_coords, center)[0][len(restaurants_coords)]-1]

# lista delle distanze in metri tra delivery points
distanze_del = tsp(delivery_points_coords, ultimo_pickup)[1]

# ultimo punto di delivery, ristorante più vicino all'ultimo punto e distanza tra i due punti
ultimo_delivery = delivery_points_coords[tsp(delivery_points_coords, center)[0][len(delivery_points_coords)]-1]
rest_piu_vicino = trova_coord_piu_vicina(restaurants_coords, ultimo_delivery)
distanza_ritorno = distance(ultimo_delivery, rest_piu_vicino)

# calcolo del tempo per ogni tratto tra i ristoranti
tempi_pickup = [distanza * (1 / velocita_media_ms) for distanza in distanze_rest]

# calcolo del tempo per ogni tratto tra i delivery points
tempi_delivery = [distanza * (1 / velocita_media_ms) for distanza in distanze_del]

# aggiunta del tempo di consegna/ritiro per ogni tratto
tempi_totali_pickup = [tempo + random_pickup_time() for tempo in tempi_pickup]
tempi_totali_delivery = [tempo + random_delivery_time() for tempo in tempi_delivery]

# somma dei tempi totali
tempo_totale_pickup = sum(tempi_totali_pickup)
tempo_totale_delivery = sum(tempi_totali_delivery)
tempo_ritorno = distanza_ritorno * (1 / velocita_media_ms)
tempo_giro_completo = tempo_totale_pickup + tempo_totale_delivery + tempo_ritorno


minuti, secondi = divmod(tempo_totale_pickup, 60)
print(f"Tempo totale pick up: {int(minuti)} minuti e {int(secondi)} secondi")

minuti, secondi = divmod(tempo_totale_delivery, 60)
print(f"Tempo totale delivery: {int(minuti)} minuti e {int(secondi)} secondi")

minuti, secondi = divmod(tempo_ritorno, 60)
print(f"Tempo totale ritorno: {int(minuti)} minuti e {int(secondi)} secondi")

minuti, secondi = divmod(tempo_giro_completo, 60)
print(f"Tempo totale del giro: {int(minuti)} minuti e {int(secondi)} secondi")

Tempo totale pick up: 13 minuti e 31 secondi
Tempo totale delivery: 31 minuti e 27 secondi
Tempo totale ritorno: 2 minuti e 45 secondi
Tempo totale del giro: 47 minuti e 44 secondi
